In [1]:
import os

import numpy as np
import pandas as pd
import keras

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

Using TensorFlow backend.


In [2]:
file = os.path.join("data", "bare_all.csv")

data = pd.read_csv(file, delimiter='\t')

In [3]:
global_label_counts = np.array(np.unique(data.label, return_counts = True))
global_label_counts

array([['cs', 'math', 'phys', 'q-bio', 'q-fin', 'stat'],
       [110296, 190564, 505925, 10578, 4623, 14565]], dtype=object)

In [4]:
class_weights = 1E4/global_label_counts[1]
class_weights

array([0.09066511931529701, 0.05247580865221133, 0.01976577555961852,
       0.9453582907922102, 2.1630975556997623, 0.6865774116031582],
      dtype=object)

In [5]:
# take a sample of records to play with
small_num = 20_000
sample = data[:small_num]

text = sample.text
label = sample.label

In [6]:
# 'cs' -> 0, ..., 'stat' -> 5

label_e = LabelEncoder()
y_train = label_e.fit_transform(label)

print( list(label_e.classes_) )
print( label_e.transform(label_e.classes_) )

#label_e.inverse_transform([0]) # array(['cs'], dtype=object)

['cs', 'math', 'phys', 'q-bio', 'q-fin', 'stat']
[0 1 2 3 4 5]


In [7]:
# mask or flag LaTeX expression with a word ' _LATEX_ '

from sklearn.base import BaseEstimator, TransformerMixin

    # self.pattern = r"(\${1,2}[\s\w\d\\,\.=\(\)*{}/\[\]^;:'`<>|%&@\"!\?~#+-]*?\${1,2})"
    # why does it differ from  r'(\$.+?\$)' ?
    
class DeLaTeX(BaseEstimator, TransformerMixin):
    
    def __init__(self, behave=None, pattern=r"(\${1,2}[\s\w\d\\,\.=\(\)*{}/\[\]^;:'`<>|%&@\"!\?~#+-]*?\${1,2})", repl = ' _LATEX_ '):
        self.repl = ' _LATEX_ '
        self.pattern = pattern
        self.behave = behave
        
        if self.behave == 'mask':
            self.repl = ' _LATEX_ '
        elif self.behave == 'flag':
            self.repl = r' _LATEX_ \1'
        return None

    def fit(self, X, y=None):
        if not isinstance(X, pd.core.series.Series):
            raise TypeError("The data must be a pandas Series of strings")
        return self

    def transform(self, X):
        if self.behave:
            return X.str.replace(self.pattern, self.repl)
        return X
            

In [8]:
text_train = text
text_train.shape

(20000,)

In [9]:
delatex = DeLaTeX(behave='mask')
tex_text_train = delatex.fit_transform(text_train)

In [10]:
tex_text_train[:3]

0    Power sum decompositions of defining equations...
1    A Spectroscopic Survey of Massive Stars in M31...
2    Hamiltonian Thermodynamics of Black Holes in G...
Name: text, dtype: object

In [11]:
tex_text_train.str.len().max()

3120

In [12]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=5000, lower=True)

In [23]:
from keras.models import Sequential
from keras import layers

Using TensorFlow backend.


In [ ]:
categorical_crossentropy

In [ ]:
model = Sequential()
model.add(layers.Dense(10, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()